# 1.0 Data Processing

In this notebook, we will discuss the key data processing steps.

## 1.1 Video to Image Decomposition

### Implementation

In [6]:
from helpers.video_utils import *

def decompose_video_to_frames(input_video_path, output_dir):
    if not is_video_openable(input_video_path):
        return False
        
    fps, frame_interval, _ = get_video_properties(input_video_path)
    image_count = process_video_images(input_video_path, output_dir, frame_interval, fps)
    print(f"Processed {image_count} frames from the video.")
    return True

## 1.2 Pose Landmark Extraction

### Implementation

In [7]:
import os
import cv2
import pandas as pd

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from helpers.file_system_utils import *
from helpers.image_utils import *

def initialize_landmarker(model_path):
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.PoseLandmarkerOptions(base_options=base_options, output_segmentation_masks=True)
    return vision.PoseLandmarker.create_from_options(options)

def populate_pose_data_with_landmarks(pose_info, landmarks):
    for idx, landmark in enumerate(landmarks):
        idx_str = str(idx).zfill(2)
        pose_info[f'landmark_{idx_str}_x'] = landmark.x
        pose_info[f'landmark_{idx_str}_y'] = landmark.y
        pose_info[f'landmark_{idx_str}_z'] = landmark.z
        pose_info[f'landmark_{idx_str}_v'] = landmark.visibility
        
def generate_pose_landmark_dictionary(source_dir, model_path, is_video=False):
    annotated_dir = create_annotated_directory(source_dir)
    filenames = get_image_filenames(source_dir)
    landmarker = initialize_landmarker(model_path)

    if is_video:
        pose_data, errors = batch_process_video_images(annotated_dir, filenames, landmarker)
    else:
        pose_data, errors = batch_process_static_images(annotated_dir, filenames, landmarker)
    
    pose_data_df = pd.DataFrame(pose_data)
    pose_data_df.to_csv(f'{source_dir}/pose_data.csv', index=False)
    
    errors = write_error_log(source_dir, errors)
    
    return print(annotated_dir)

def batch_process_video_images(annotated_dir, filenames, landmarker):
    pose_data = []
    errors = []

    for image_file_path in sorted(filenames):
        
        image_bgr = cv2.imread(image_file_path, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

        image_filename, second, frame_no = parse_video_filename(image_file_path)
        detection_result = landmarker.detect(mp_image)

        if detection_result.pose_landmarks:
            annotate_and_save_image(annotated_dir, image_filename, detection_result, image_rgb, scale=0.4)
            
            for landmarks in detection_result.pose_landmarks:
                pose_info = {
                    'image_filename': image_filename,
                    'secs': second,
                    'frame_no': frame_no
                }
                populate_pose_data_with_landmarks(pose_info, landmarks)
            pose_data.append(pose_info)
        else:
            errors.append(image_file_path)
    
    return pose_data, errors

def batch_process_static_images(annotated_dir, filenames, landmarker):
    
    pose_data = []
    errors = []

    for image_file_path in sorted(filenames):
        
        image_bgr = cv2.imread(image_file_path, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

        image_filename = os.path.basename(image_file_path)        
        pose_name = image_filename.split('.')[0]
        
        detection_result = landmarker.detect(mp_image)
        
        if detection_result.pose_landmarks:
            annotate_and_save_image(annotated_dir, image_filename, detection_result, image_rgb, scale=1)
            
            for landmarks in detection_result.pose_landmarks:
                pose_info = {
                    'image_filename': image_filename,
                    'pose_name': pose_name
                }
                populate_pose_data_with_landmarks(pose_info, landmarks)
            pose_data.append(pose_info)
        else:
            errors.append(image_file_path)
    
    return pose_data, errors

# Example Usage

In [12]:
model_path = 'models/ml/pose_landmarker.task'

### Video Processing

In [15]:
input_video_path = 'data/ground_truth/uploads/combined.mp4'
output_dir = 'data/ground_truth/processed/combined/'

# 1.1 Video to Image Decomposition
decompose_video_to_frames(input_video_path, output_dir)

# 1.2 Pose Landmark Extraction
generate_pose_landmark_dictionary(source_dir=output_dir,model_path=model_path,is_video=True)

Processed 220 frames from the video.
data/ground_truth/processed/combined/annotated


### Pose Landmark Extract for Static Images

In [5]:
dict_source_dir = 'data/external/positions/body/'
generate_pose_landmark_dictionary(source_dir=dict_source_dir,model_path=model_path,is_video=False)

dict_source_dir = 'data/external/positions/legs/'
generate_pose_landmark_dictionary(source_dir=dict_source_dir,model_path=model_path,is_video=False)

dict_source_dir = 'data/external/positions/grips/'
generate_pose_landmark_dictionary(source_dir=dict_source_dir,model_path=model_path,is_video=False)

data/external/positions/body/annotated
data/external/positions/legs/annotated
data/external/positions/grips/annotated
